# MET 585 (Cloud Physics) Course Project (NIU)
## An Analysis of the Climatological Trend in Mid-Latitude Baroclinity
## Robert C. Fritzen

### Synopsis
Barocliic Instability (Baroclinity) is one of the fundamental foci of the development of extratropical cyclones in the mid-latitudes. Defined as the growth of small scale disturbances through time, baroclinity is expressed as a reinforcing interaction of wind shear at different levels of the atmosphere. As the Thermal Wind relationship connects surface temperature gradients to wind shear, we can define baroclinity using the Brunt-Väisälä Frequency:

<font size=4>
$\sigma_{BI} = \frac{f}{N}\frac{d\vec{V}}{dZ}$
</font>

Which defines: The coriolis parameter (f), the Brunt-Väisälä Oscillation Frequency (N), and the wind shear vector with respect to geopotential height ($\frac{d\vec{V}}{dZ}$)

Using the definition of the Brunt-Väisälä Oscillation Frequency, $N = \sqrt{\frac{g}{\theta}\frac{\partial\theta}{{\partial}Z}}$, The previous formula can be expanded to:

<font size=4>
$\sigma_{BI} = \frac{f}{\sqrt{\frac{g}{{\theta}_m}\frac{{\theta}_u - {\theta}_l}{Z_u - Z_l}}}\frac{d\vec{V}}{dZ} $
</font>

Which defines a change in the potential temperature (${\theta}$), with respect to height. By differential approximation methods, we analyze this as a change between the upper and lower analysis levels.

This project seeks to explore the climatological trend in baroclinity over the past 70 years. Using the NCEP/NCAR Reanalysis Data, baroclinity can be calculated at three different levels (Low, Middle, and Upper) and then explored using statisical tools such as standard anomaly and moving averages to explore how baroclinity is changing with time.

It is hypothesized that under climate change scenarios, the meridional (Y) temperature gradient is weakening, and hence the low level temperature gradient is also weakening, reducing baroclinity with time.

### Python
This notebook was written with Anaconda 2, using various scientific analysis packages. To run this notebook you will need to install the following packages:

* numpy
* matplotlib
* Basemap
* scipy (stats module)
* netCDF4
* ftplib

This notebook was constructed with sequential execution in mind. Run the code blocks in the order they are presented as each block requires the next in order to execute completely, this was done for organizational purposes and to keep things as neat as possible for readability and replication purposes.

In [1]:
# Required Packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import scipy.stats as stats
from netCDF4 import Dataset
import os
from ftplib import FTP
import calendar

In [3]:
## Constants and Variables

# Plotting Commands
make_average_monthly_plots = True
make_average_yearly_plots = True
make_difference_plots = True
make_mean_compare_plots = True
make_standard_anomaly_plots = True
make_moving_average_plots = True
make_moving_difference_plots = True

# Storage Variables
storePath = "D:/Robert Docs/College/NIU/MET 585/Project/"
dataPath = storePath + "Data/"
plotPath = storePath + "Plots/"

p_anom_low = plotPath + "anom_low/"
p_amon_mid = plotPath + "anom_mid/"
p_anom_up = plotPath + "anom_up/"
p_avg_low_mo = plotPath + "avg_low_month/"
p_avg_mid_mo = plotPath + "avg_mid_month/"
p_avg_up_mo = plotPath + "avg_up_month/"
p_avg_low_ye = plotPath + "low_year/"
p_avg_mid_ye = plotPath + "mid_year/"
p_avg_up_ye = plotPath + "up_year/"
p_d_low_mo = plotPath + "d_low_month/"
p_d_mid_mo = plotPath + "d_mid_month/"
p_d_up_mo = plotPath + "d_up_month/"
p_d_low_ye = plotPath + "d_low_year/"
p_d_mid_ye = plotPath + "d_mid_year/"
p_d_up_ye = plotPath + "d_up_year/"
p_mean_c_l = plotPath + "low_mean_comp/"
p_mean_c_m = plotPath + "mid_mean_comp/"
p_mean_c_u = plotPath + "up_mean_comp/"
p_mov_low = plotPath + "moving_avg_low/"
p_mov_mid = plotPath + "moving_avg_mid/"
p_mov_up = plotPath + "moving_avg_up/"
p_dmov_low = plotPath + "d_moving_avg_low/"
p_dmov_mid = plotPath + "d_moving_avg_mid/"
p_dmov_up = plotPath + "d_moving_avg_up/"

monthPaths = ["january/", "february/", "march/", "april/", "may/", 
              "june/", "july/", "august/", "september/", "october/", 
              "november/", "december/"]

#ensoTable: Defines the ENSO phase for that year (-1: LAN, 0: NEU, 1: ELN), note 1948-49 are not present in the CPC
#            records, so they are assumed neutral
ensoTable = [0, 0, -1, 1, 0, 1, -1, -1, -1, 1, 1, 0, 0, 0, 0, 1, 
            #1964
             -1, 1, 0, 0, 0, 1, -1, -1, 1, -1, -1, -1, 0, 1, 0, 0,
            #1980
            0, 0, 1, 1, 0, -1, 0, 1, -1, -1, 0, 1, 1, 0, 1, -1, -1,
            #1997
            1, 1, -1, -1, 0, 1, 0, 1, 0, 0, -1, -1, 1, -1, -1, 0,
            #2013
            0, 0, 1, 1, 0]

linkBase = "ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis/pressure/" #air.yyyy.nc, hgt.yyyy.nc, uwnd.yyyy.nc, vwnd.yyyy.nc
years = np.arange(1948, 2018, 1) # Stop at 2017, 2018 is "in progress", so we'll exclude
hoursPerDay = 4 # Four daily observations per ncep file (0z, 6z, 12z, and 18z)

# Meteorological Constants
Omega = 7.29 * (10**-5)
TwoOmega = 2 * Omega
ROverCp = 287./1004
gConst = 9.81

In [12]:
## Required Functions

# calc_theta(T, P):
## Calculate potential temperature using poisson's equation
## Variables: T - Temperature (Kelvin), P - Pressure (Pascals)
## Returns: Theta - Potential Temperature (Kelvin)
def calc_theta(T, P):
    return T * (1000 / P) ** ROverCp

# Functions
def paired_t_test(a, b):
    at = np.array(a)
    bt = np.array(b)

    y,x = at.shape
    stat = np.zeros((y, x))
    pvalue = np.zeros((y, x))
    for yy in range(y):
        for xx in range(x):
            stat[yy,xx],pvalue[yy,xx] = stats.ttest_ind(at[yy,xx], bt[yy,xx], nan_policy="omit")
    #stat,pvalue = stats.ttest_ind(at, bt, nan_policy="omit")
    return tuple((stat,pvalue)) 

# prepareMap()
## Generate a basemap plot with the plotting location defined.
## Returns: m - Basemap object 
def prepareMap():
    plt.figure()
    m = Basemap(projection='mill',llcrnrlon=0,llcrnrlat=10,urcrnrlon=360,urcrnrlat=80)
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    m.drawmapboundary()
    return m

def makeBCIPlot(baroclinic, title, savePath):
    BI_F=np.flipud(baroclinic)
    m = prepareMap()

    pRange = np.linspace(1e-5, 4, 40, endpoint=True)
    #cs = m.contour(lon, lat, BI, pRange, linewidths=0.5, colors='k')
    ny = BI_F.shape[0] 
    nx = BI_F.shape[1]
    lons, lats = m.makegrid(nx, ny)
    x, y = m(lons, lats)
    cs = m.contourf(x, y, BI_F, pRange, cmap=plt.cm.jet)
    cbar = m.colorbar(cs, location='bottom', pad="5%")
    cbar.set_label('s^-1 (E-6)')    
    plt.suptitle(title)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    plt.savefig(savePath, bbox_inches='tight')
    plt.close()
    print("Plot saved: " + savePath)
    
#d_Plot(): 2 - 1 (Red: Decrease, Blue: Increase)
def make_D_BCIPlot(baroclinic1, baroclinic2, title, savePath):
    BI_1=np.flipud(baroclinic1)
    BI_2=np.flipud(baroclinic2)
    BI_F = BI_2 - BI_1
    m = prepareMap()

    pRange = np.linspace(-0.5, 0.5, 40, endpoint=True)
    #cs = m.contour(lon, lat, BI, pRange, linewidths=0.5, colors='k')
    ny = BI_F.shape[0] 
    nx = BI_F.shape[1]
    lons, lats = m.makegrid(nx, ny)
    x, y = m(lons, lats)
    cs = m.contourf(x, y, BI_F, pRange, cmap=plt.cm.RdBu)
    cbar = m.colorbar(cs, location='bottom', pad="5%")
    cbar.set_label('s^-1 (E-6)')    
    plt.suptitle(title)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    plt.savefig(savePath, bbox_inches='tight')
    plt.close()
    print("Plot saved: " + savePath)    

def make_anom_BCIPlot(stdAnomaly, title, savePath):
    BI_F=np.flipud(stdAnomaly)
    m = prepareMap()

    pRange = np.linspace(-3, 3, 40, endpoint=True)
    #cs = m.contour(lon, lat, BI, pRange, linewidths=0.5, colors='k')
    ny = BI_F.shape[0] 
    nx = BI_F.shape[1]
    lons, lats = m.makegrid(nx, ny)
    x, y = m(lons, lats)
    cs = m.contourf(x, y, BI_F, pRange, cmap=plt.cm.RdBu)
    cbar = m.colorbar(cs, location='bottom', pad="5%")
    cbar.set_label('s^-1 (E-6)')    
    plt.suptitle(title)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    plt.savefig(savePath, bbox_inches='tight')
    plt.close()
    print("Plot saved: " + savePath)
    
def make_mean_compare_BCIPlot(mean, baroclinic, title, savePath, withStats = True):    
    BI_1=np.flipud(mean)
    BI_2=np.flipud(baroclinic)
    BI_F = BI_2 - BI_1 
    
    m = prepareMap()

    pRange = np.linspace(-1, 1, 40, endpoint=True)
    #cs = m.contour(lon, lat, BI, pRange, linewidths=0.5, colors='k')
    ny = BI_F.shape[0] 
    nx = BI_F.shape[1]
    lons, lats = m.makegrid(nx, ny)
    x, y = m(lons, lats)
    cs = m.contourf(x, y, BI_F, pRange, cmap=plt.cm.RdBu)
    if(withStats):
        tStat, pvalue = paired_t_test(BI_1, BI_2)
        critu1cs = plt.contour(x, y, tStat, levels=[2,977], cmap=plt.get_cmap('gray'), linestyles='dashed')
    
    cbar = m.colorbar(cs, location='bottom', pad="5%")
    cbar.set_label('s^-1 (E-6)')    
    plt.suptitle(title)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    plt.savefig(savePath, bbox_inches='tight')
    plt.close()
    print("Plot saved: " + savePath)     

## FTP Code Block
The following code block will download the NCEP/NCAR Reanalysis Files from the CDC NOAA Page (ftp.cdc.noaa.gov). This server accepts anonomous logins, and therefore we can use the FTPLib package to download the files. I chose this over urllib as urllib has a caching problem after downloading a single file from FTP servers that can cause additional downloads to corrupt. As we're dowloading 4 * n(years) files, that are each ~500MB large, this can be problematic.

The file storage for our particular dataset is located in: /Datasets/ncep.reanalysis/pressure/ and has a naming format as such:

* Air Temperature: air.yyyy.nc
* Geopotential Height: hgt.yyyy.nc
* Zonal Wind: uwnd.yyyy.nc
* Meridional Wind: vwnd.yyyy.nc

Once the four files for a single year are collected, baroclinity is calculated for SFC-850mb, 850mb - 500mb, and 700mb - 300mb and stored into a single NetCDF file named baroclinic.yyyy.nc in the same directory. The original NCEP files are then removed from your computer to preserve storage space (~1.5GB for the four files vs ~90MB for the single baroclinic file)

**NOTE: Running this code block will take A LONG TIME, you are downloading 70 years of data and performing calculations on these data as the downloads are being completed, it took my laptop about 15 hours to complete the full download and calculation processing**

In [ ]:
ftp = FTP('ftp.cdc.noaa.gov')
ftp.login()

print("Beginning collection of NETCDF files (THIS WILL TAKE A LONG TIME)")
for y in years:
    print("Beginning Collection for " + str(y))
    tLink = "/Datasets/ncep.reanalysis/pressure/air." + str(y) + ".nc"
    hLink = "/Datasets/ncep.reanalysis/pressure/hgt." + str(y) + ".nc"
    uLink = "/Datasets/ncep.reanalysis/pressure/uwnd." + str(y) + ".nc"
    vLink = "/Datasets/ncep.reanalysis/pressure/vwnd." + str(y) + ".nc" 
    
    print("Downloading air temperature, storing in tmp.air.nc")
    
    localfile1 = open(dataPath + "tmp.air.nc", 'wb')
    ftp.retrbinary('RETR ' + tLink, localfile1.write, 1024)
    localfile1.close()    
    
    print("Downloading height, storing in tmp.hgt.nc")
    localfile2 = open(dataPath + "tmp.hgt.nc", 'wb')
    ftp.retrbinary('RETR ' + hLink, localfile2.write, 1024)
    localfile2.close() 
    
    print("Downloading u winds, storing in tmp.uwnd.nc")
    localfile3 = open(dataPath + "tmp.uwnd.nc", 'wb')
    ftp.retrbinary('RETR ' + uLink, localfile3.write, 1024)
    localfile3.close() 
    
    print("Downloading v winds, storing in tmp.vwnd.nc")
    localfile4 = open(dataPath + "tmp.vwnd.nc", 'wb')
    ftp.retrbinary('RETR ' + vLink, localfile4.write, 1024)
    localfile4.close() 
    
    print("Downloads complete, opening netCDF files")
    
    aTmp = Dataset(dataPath + "tmp.air.nc")
    hTmp = Dataset(dataPath + "tmp.hgt.nc")
    uTmp = Dataset(dataPath + "tmp.uwnd.nc")
    vTmp = Dataset(dataPath + "tmp.vwnd.nc")
    
    outFile = dataPath + "baroclinic." + str(y) + ".nc"
    newNCFile = Dataset(outFile, 'w', format='NETCDF4_CLASSIC')
    time = newNCFile.createDimension('time', size=None) 
    lat = newNCFile.createDimension('lat', 33)
    lon = newNCFile.createDimension('lon', 144)  
    
    ll_BCI = newNCFile.createVariable('BCI_Low', np.float32, ('time','lat','lon'))
    ml_BCI = newNCFile.createVariable('BCI_Mid', np.float32, ('time','lat','lon'))   
    ul_BCI = newNCFile.createVariable('BCI_Hi', np.float32, ('time','lat','lon'))   
    
    print("Begin Baroclinic Calculations")
    
    # Note, we only want latitudes between 5N and 85N ( 2->35 )
    T = np.squeeze(aTmp.variables['air'][:,:,2:35,:]) 
    U = np.squeeze(uTmp.variables['uwnd'][:,:,2:35,:])
    V = np.squeeze(vTmp.variables['vwnd'][:,:,2:35,:])
    H = np.squeeze(hTmp.variables['hgt'][:,:,2:35,:])
    wVel = (U**2 + V**2)**0.5
    
    lat = np.linspace(5,85,33)
    lon = np.linspace(-177.5,180,144)
    p = aTmp.variables['level'][:] # Grab the isobaric levels
    
    itime = len(aTmp.variables['time'][:]) 
    ilat = len(lat)
    ilon = len(lon)    
    
    f = TwoOmega * np.sin(lat * (np.pi / 180))
    # Construct a "coriolis" array of equivalent size to the other arrays
    corPar = np.zeros((itime, 1, 1, ilat, ilon))
    for i,l in enumerate(lat):
        corPar[:,0,0,i,:] = f[np.where(lat == l)]    
    
    print("Calculating Low Level Baroclinity")
    # Low Level: 1000, 850, 700
    low = 1000
    med = 850
    high = 700
    
    wVelLow = wVel[:,np.where(p==low),:,:]
    wVelHigh = wVel[:,np.where(p==high),:,:]  
    geoHgtLow = H[:,np.where(p==low),:,:]
    geoHgtHigh = H[:,np.where(p==high),:,:]    
    thetaLow = calc_theta(T[:,np.where(p==low),:,:], low)
    thetaMid = calc_theta(T[:,np.where(p==med),:,:], med)
    thetaHi = calc_theta(T[:,np.where(p==high),:,:], high)
    
    dTheta = thetaHi - thetaLow
    dZ = geoHgtHigh - geoHgtLow
    rootTerm = np.sqrt((gConst / thetaMid) * (dTheta / dZ))
    outerTerm = (wVelHigh - wVelLow) / dZ  
    BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000    
    ll_BCI[:] = BI[:,0,0,:,:]   
    
    print("Calculating Mid Level Baroclinity")
    # Mid Level: 850, 700, 500
    low = 850
    med = 700
    high = 500
    
    wVelLow = wVel[:,np.where(p==low),:,:]
    wVelHigh = wVel[:,np.where(p==high),:,:]  
    geoHgtLow = H[:,np.where(p==low),:,:]
    geoHgtHigh = H[:,np.where(p==high),:,:]    
    thetaLow = calc_theta(T[:,np.where(p==low),:,:], low)
    thetaMid = calc_theta(T[:,np.where(p==med),:,:], med)
    thetaHi = calc_theta(T[:,np.where(p==high),:,:], high)
    
    dTheta = thetaHi - thetaLow
    dZ = geoHgtHigh - geoHgtLow
    rootTerm = np.sqrt((gConst / thetaMid) * (dTheta / dZ))
    outerTerm = (wVelHigh - wVelLow) / dZ
    BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000
    
    ml_BCI[:] = BI[:,0,0,:,:]       
    
    print("Calculating Upper Level Baroclinity")
    # Upper Level: 700, 500, 300
    low = 700
    med = 500
    high = 300
    
    wVelLow = wVel[:,np.where(p==low),:,:]
    wVelHigh = wVel[:,np.where(p==high),:,:]  
    geoHgtLow = H[:,np.where(p==low),:,:]
    geoHgtHigh = H[:,np.where(p==high),:,:]    
    thetaLow = calc_theta(T[:,np.where(p==low),:,:], low)
    thetaMid = calc_theta(T[:,np.where(p==med),:,:], med)
    thetaHi = calc_theta(T[:,np.where(p==high),:,:], high)
    
    dTheta = thetaHi - thetaLow
    dZ = geoHgtHigh - geoHgtLow
    rootTerm = np.sqrt((gConst / thetaMid) * (dTheta / dZ))
    outerTerm = (wVelHigh - wVelLow) / dZ    
    BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000
    
    ul_BCI[:] = BI[:,0,0,:,:]      
    
    print("Saving output NetCDF file baroclinic." + str(y) + ".nc")

    aTmp.close()
    hTmp.close()
    uTmp.close()
    vTmp.close()  
    newNCFile.close()
    
    print("Deleting temporary files")
    os.remove(dataPath + "tmp.air.nc")
    os.remove(dataPath + "tmp.hgt.nc")
    os.remove(dataPath + "tmp.uwnd.nc")
    os.remove(dataPath + "tmp.vwnd.nc")

ftp.quit()
    
print("Data generation completed.")

## Pre-Analysis Calculation Code Block
This code block handles all of the pre-analysis calculations. This mainly accesses the NetCDF files and sorts/stores the data into dictionaries that can be easily accessed in analysis steps for quick calculations and plotting routines.

In [ ]:
monthAnalysis = {}
yearAnalysis = {}

tmp_ll_mo = np.zeros((33,144))
tmp_ml_mo = np.zeros((33,144))
tmp_ul_mo = np.zeros((33,144))

tmp_ll_yr = np.zeros((33,144))
tmp_ml_yr = np.zeros((33,144))
tmp_ul_yr = np.zeros((33,144))

print("Begin pre-analysis calculations")

print("Calculating monthly and annual means")
for y in years:
    print("Opening: " + dataPath + "baroclinic." + str(y) + ".nc")
    ncFile = Dataset(dataPath + "baroclinic." + str(y) + ".nc")
    llOb = ncFile.variables['BCI_Low'][:,:,:]
    mlOb = ncFile.variables['BCI_Mid'][:,:,:]
    ulOb = ncFile.variables['BCI_Hi'][:,:,:]

    llOb[np.isnan(llOb)] = 0
    mlOb[np.isnan(mlOb)] = 0
    ulOb[np.isnan(ulOb)] = 0        
    
    startPoint = 0
    for m in range(1, 13):
        mr = calendar.monthrange(y, m)       
        if(m > 1):
            mr_prev = calendar.monthrange(y, m-1)
            dayCount = mr_prev[1]
            startPoint += dayCount * hoursPerDay
        
        daysInMonth = mr[1]
        endPoint = hoursPerDay * daysInMonth
        obRange = range(startPoint, startPoint + endPoint)
        obCount = len(obRange)    
         
        # Calculate Average for the Month
        avg_ll = np.sum(llOb[startPoint:startPoint+endPoint,:,:], axis=0)
        avg_ll = avg_ll / obCount
        avg_ml = np.sum(mlOb[startPoint:startPoint+endPoint,:,:], axis=0)
        avg_ml = avg_ml / obCount
        avg_ul = np.sum(ulOb[startPoint:startPoint+endPoint,:,:], axis=0)
        avg_ul = avg_ul / obCount
        
        key = "low_month_" + str(m) + "_" + str(y)
        monthAnalysis[key] = avg_ll
        key = "mid_month_" + str(m) + "_" + str(y)
        monthAnalysis[key] = avg_ml
        key = "up_month_" + str(m) + "_" + str(y)
        monthAnalysis[key] = avg_ul    
        
    # Perform the Yearly Analysis
    totalYObj = len(ncFile.dimensions["time"])
    avg_ll_y = np.sum(llOb[:,:,:], axis=0)
    avg_ll_y = avg_ll_y / totalYObj
    avg_ml_y = np.sum(mlOb[:,:,:], axis=0)
    avg_ml_y = avg_ml_y / totalYObj
    avg_ul_y = np.sum(ulOb[:,:,:], axis=0)
    avg_ul_y = avg_ul_y / totalYObj  
    
    key = "low_yr_" + str(y)
    yearAnalysis[key] = avg_ll_y
    key = "mid_yr_" + str(y)
    yearAnalysis[key] = avg_ml_y
    key = "up_yr_" + str(y)
    yearAnalysis[key] = avg_ul_y
    
    ncFile.close()
    
print("Calculating Standard Mean (1980 - 2010)")
standardMean = {}
tmp1 = np.zeros((33,144))
tmp2 = np.zeros((33,144))
tmp3 = np.zeros((33,144))

sT1 = []
sT2 = []
sT3 = []
for i in range(1980, 2011):
    tmp1 += yearAnalysis["low_yr_" + str(i)]
    tmp2 += yearAnalysis["mid_yr_" + str(i)]
    tmp3 += yearAnalysis["up_yr_" + str(i)]
    
    sT1.append(yearAnalysis["low_yr_" + str(i)])
    sT2.append(yearAnalysis["mid_yr_" + str(i)])
    sT3.append(yearAnalysis["up_yr_" + str(i)])
    
sigma_low = np.std(sT1)
sigma_mid = np.std(sT2)
sigma_hi = np.std(sT3)

standardMean["low"] = tmp1 / 30
standardMean["mid"] = tmp2 / 30
standardMean["up"] = tmp3 / 30    
    
print("Done.")

## Analysis Code Block
This code block handles all of our analysis functions. Each code block will generate plots that will go into the defined plotting folder from the above constants code block.

You can control which analysis(es) will be performed by modifying the boolean flags in the second code block near the top of this notebook, a quick rundown of what each flag does follows:

* make_average_monthly_plots: Average monthly plots will calculate the average of the baroclinity for each month (IE: Dec 1 - Dec 31) in evey year and save it to the plotting folder.
* make_average_yearly_plots: Average annual (Yearly) plots will calculate the average of the bacolinity for the full year (Jab 1 - Dec 31) and save the plots to the yearly plotting folder.
* make_difference_plots: Difference plots will make two pairs of differences. The first are monthly differences, which compare one month's average to the same month in the previous year (IE: Jan 1950 - Jan 1949). The other is annual difference which compares one year's average to the previous (IE 1950 - 1949).
* make_mean_compare_plots: This tool will compute the standard mean (Average baroclinity for 1980 - 2010) and then generate yearly difference plots between a given year's mean and the standard mean.
* make_standard_anomaly_plots: This tool will calculate the standard anomaly between each year and the standard mean period (1980 - 2010), for more information on this, see: http://www.wpc.ncep.noaa.gov/hmt/ohrfc_training.ppt
* make_moving_average_plots: Moving averages computes a thirty year average for one period and then moves forward to the next thirty years At the moment, only two of these can be generated due to the size of the data set.
* make_moving_difference_plots: Moving differences performs the same moving average computation from the prior, but calculates the difference between these averages.

Some of these processes are computationally expensive and will take some time to process so make sure to set the flag variables above prior to running this block.

In [ ]:
if(make_average_yearly_plots or make_average_monthly_plots):
    for y in years:
        if(make_average_monthly_plots):
            for m in range(1, 13):             
                mStr = str("0") + str(m) if m < 10 else str(m)
                plotTitle1 = "Average Low-Level Baroclinity (" + mStr + " / " + str(y) + ")"
                plotPath1 = p_avg_low_mo + monthPaths[m-1] + "low_" + mStr + "-" + str(y) + ".png"
                key = "low_month_" + str(m) + "_" + str(y)
                avg_ll = monthAnalysis[key]
                makeBCIPlot(avg_ll, plotTitle1, plotPath1)
                
                plotTitle1 = "Average Mid-Level Baroclinity (" + mStr + " / " + str(y) + ")"
                plotPath1 = p_avg_mid_mo + monthPaths[m-1] + "mid_" + mStr + "-" + str(y) + ".png"
                key = "mid_month_" + str(m) + "_" + str(y)
                avg_ml = monthAnalysis[key]                
                makeBCIPlot(avg_ml, plotTitle1, plotPath1)
                
                plotTitle1 = "Average Upper-Level Baroclinity (" + mStr + " / " +  str(y) + ")"
                plotPath1 = p_avg_up_mo + monthPaths[m-1] + "upper_" + mStr + "-" + str(y) + ".png"
                key = "up_month_" + str(m) + "_" + str(y)
                avg_ul = monthAnalysis[key]                 
                makeBCIPlot(avg_ul, plotTitle1, plotPath1)           

        if(make_average_yearly_plots):
            key = "low_yr_" + str(y)
            avg_ll_y = yearAnalysis[key]
            key = "mid_yr_" + str(y)
            avg_ml_y = yearAnalysis[key] 
            key = "up_yr_" + str(y)
            avg_ul_y = yearAnalysis[key]            
            
            plotTitle1 = "Average Low-Level Baroclinity (" + str(y) + ")"
            plotPath1 = p_avg_low_ye + "low_y-" + str(y) + ".png"
            makeBCIPlot(avg_ll_y, plotTitle1, plotPath1)
            plotTitle1 = "Average Mid-Level Baroclinity (" + str(y) + ")"
            plotPath1 = p_avg_mid_ye + "mid_y-" + str(y) + ".png"
            makeBCIPlot(avg_ml_y, plotTitle1, plotPath1)
            plotTitle1 = "Average Upper-Level Baroclinity (" +  str(y) + ")"
            plotPath1 = p_avg_up_ye + "upper_y-" + str(y) + ".png"
            makeBCIPlot(avg_ul_y, plotTitle1, plotPath1)    
    
# Analysis 1/2: Difference Plots
# D_Month
if(make_difference_plots):
    for m in range(1, 13):
        for y in years:
            if(y > 1948):
                prior_low = "low_month_" + str(m) + "_" + str(y-1)
                now_low = "low_month_" + str(m) + "_" + str(y)
                prior_mid = "mid_month_" + str(m) + "_" + str(y-1)
                now_mid = "mid_month_" + str(m) + "_" + str(y)
                prior_up = "up_month_" + str(m) + "_" + str(y-1)
                now_up = "up_month_" + str(m) + "_" + str(y)     

                bl_prior = monthAnalysis[prior_low]
                bl_now = monthAnalysis[now_low]
                bm_prior = monthAnalysis[prior_mid]
                bm_now = monthAnalysis[now_mid]
                bu_prior = monthAnalysis[prior_up]
                bu_now = monthAnalysis[now_up]  

                pathLow = p_d_low_mo + monthPaths[m-1] + "d_low_" + str(m) + "_" + str(y-1) + "-" + str(y) + ".png"
                plotTitle1 = "Low-Level Baroclinity Difference (" + str(m) + "/" + str(y) + " - " + str(m) + "/" + str(y-1) + ")"
                make_D_BCIPlot(bl_prior, bl_now, plotTitle1, pathLow)
                pathLow = p_d_mid_mo + monthPaths[m-1] + "d_mid_" + str(m) + "_" + str(y-1) + "-" + str(y) + ".png"
                plotTitle1 = "Mid-Level Baroclinity Difference (" + str(m) + "/" + str(y) + " - " + str(m) + "/" + str(y-1) + ")"
                make_D_BCIPlot(bm_prior, bm_now, plotTitle1, pathLow)
                pathLow = p_d_up_mo + monthPaths[m-1] + "d_up_" + str(m) + "_" + str(y-1) + "-" + str(y) + ".png"
                plotTitle1 = "Upper-Level Baroclinity Difference (" + str(m) + "/" + str(y) + " - " + str(m) + "/" + str(y-1) + ")"
                make_D_BCIPlot(bu_prior, bu_now, plotTitle1, pathLow)                   
    
# D_Year    
    for y in years:
        if(y > 1948):
            prior_low = "low_yr_" + str(y-1)
            now_low = "low_yr_" + str(y)
            prior_mid = "mid_yr_" + str(y-1)
            now_mid = "mid_yr_" + str(y)
            prior_up = "up_yr_" + str(y-1)
            now_up = "up_yr_" + str(y)

            bl_prior = yearAnalysis[prior_low]
            bl_now = yearAnalysis[now_low]
            bm_prior = yearAnalysis[prior_mid]
            bm_now = yearAnalysis[now_mid]
            bu_prior = yearAnalysis[prior_up]
            bu_now = yearAnalysis[now_up]

            pathLow = p_d_low_ye + "d_low_" + str(y-1) + "-" + str(y) + ".png"
            plotTitle1 = "Low-Level Baroclinity Difference (" + str(y) + " - " + str(y-1) + ")"
            make_D_BCIPlot(bl_prior, bl_now, plotTitle1, pathLow)
            pathLow = p_d_mid_ye + "d_mid_" + str(y-1) + "-" + str(y) + ".png"
            plotTitle1 = "Mid-Level Baroclinity Difference (" + str(y) + " - " + str(y-1) + ")"
            make_D_BCIPlot(bm_prior, bm_now, plotTitle1, pathLow)
            pathLow = p_d_up_ye + "d_up_" + str(y-1) + "-" + str(y) + ".png"
            plotTitle1 = "Upper-Level Baroclinity Difference (" + str(y) + " - " + str(y-1) + ")"
            make_D_BCIPlot(bu_prior, bu_now, plotTitle1, pathLow)            
    
# Analysis 3:
# Anomaly Plots (Comparing to 1980 - 2010 "Mean")


for y in years:
    low = yearAnalysis["low_yr_" + str(y)]
    mid = yearAnalysis["mid_yr_" + str(y)]
    up = yearAnalysis["up_yr_" + str(y)]
    
    if(make_mean_compare_plots):
        plotTitle = "Difference of " + str(y) + " Low-Level Baroclinity to 1980-2010 Mean"# (Dashed Significant to 1%)" 
        savePath = p_mean_c_l + "low_mean_comp_" + str(y) + ".png"
        make_mean_compare_BCIPlot(standardMean["low"], low, plotTitle, savePath, withStats=False)
        plotTitle = "Difference of " + str(y) + " Mid-Level Baroclinity to 1980-2010 Mean"# (Dashed Significant to 1%)" 
        savePath = p_mean_c_m + "mid_mean_comp_" + str(y) + ".png"
        make_mean_compare_BCIPlot(standardMean["mid"], mid, plotTitle, savePath, withStats=False)
        plotTitle = "Difference of " + str(y) + " Upper-Level Baroclinity to 1980-2010 Mean"# (Dashed Significant to 1%)" 
        savePath = p_mean_c_u + "up_mean_comp_" + str(y) + ".png"
        make_mean_compare_BCIPlot(standardMean["up"], up, plotTitle, savePath, withStats=False)
    
    if(make_standard_anomaly_plots):
        anom_low = (low - standardMean["low"]) / sigma_low
        anom_mid = (mid - standardMean["mid"]) / sigma_mid
        anom_up = (up - standardMean["up"]) / sigma_hi

        plotTitle = "Standard Anomaly of Low-Level Baroclinity (" + str(y) + ") (Comp. 1980-2010 Mean)"
        plotSavePath = p_anom_low + "low_anom_" + str(y) + ".png"
        make_anom_BCIPlot(anom_low, plotTitle, plotSavePath)
        plotTitle = "Standard Anomaly of Mid-Level Baroclinity (" + str(y) + ") (Comp. 1980-2010 Mean)"
        plotSavePath = p_anom_mid + "mid_anom_" + str(y) + ".png"
        make_anom_BCIPlot(anom_mid, plotTitle, plotSavePath)
        plotTitle = "Standard Anomaly of Upper-Level Baroclinity (" + str(y) + ") (Comp. 1980-2010 Mean)"
        plotSavePath = p_anom_up + "up_anom_" + str(y) + ".png"
        make_anom_BCIPlot(anom_up, plotTitle, plotSavePath) 
        
# Analysis 4: Moving Averages
if(make_moving_average_plots):
    for m in range(1, 13):
        mStr = str("0") + str(m) if m < 10 else str(m)
        for y in years:
            if((y + 30) < 2018):
                totl = np.zeros((33,144))
                totm = np.zeros((33,144))
                totu = np.zeros((33,144))
                for i in range(y, y+31):
                    totl += monthAnalysis["low_month_" + str(m) + "_" + str(i)]
                    totm += monthAnalysis["mid_month_" + str(m) + "_" + str(i)]
                    totu += monthAnalysis["up_month_" + str(m) + "_" + str(i)]
                # Compute Average
                totl /= 30
                totm /= 30
                totu /= 30
                # Plot
                plotTitle1 = "Moving Average Low-Level Baroclinity ((" + mStr + ") - " + str(y) + " - " + str(y+30) + ")"
                plotPath1 = p_mov_low + monthPaths[m-1] + "mavg_low_" + mStr + "-" + str(y) + "-" + str(y+30) + ".png"
                makeBCIPlot(totl, plotTitle1, plotPath1)            
                plotTitle1 = "Moving Average Mid-Level Baroclinity ((" + mStr + ") - " + str(y) + " - " + str(y+30) + ")"
                plotPath1 = p_mov_mid + monthPaths[m-1] + "mavg_mid_" + mStr + "-" + str(y) + "-" + str(y+30) + ".png"
                makeBCIPlot(totm, plotTitle1, plotPath1)     
                plotTitle1 = "Moving Average Upper-Level Baroclinity ((" + mStr + ") - " + str(y) + " - " + str(y+30) + ")"
                plotPath1 = p_mov_up + monthPaths[m-1] + "mavg_up_" + mStr + "-" + str(y) + "-" + str(y+30) + ".png"
                makeBCIPlot(totu, plotTitle1, plotPath1)      
                
if(make_moving_difference_plots):
    for m in range(1, 13):
        mStr = str("0") + str(m) if m < 10 else str(m)
        y = years[0]
        while(True):
            print("Plotting for " + str(y))
            if((y + 20) < 2018-20):
                totl_before = np.zeros((33,144))
                totm_before = np.zeros((33,144))
                totu_before = np.zeros((33,144))                
                totl_after = np.zeros((33,144))
                totm_after = np.zeros((33,144))
                totu_after = np.zeros((33,144)) 
                
                for i in range(y, y+21):
                    totl_before += monthAnalysis["low_month_" + str(m) + "_" + str(i)]
                    totm_before += monthAnalysis["mid_month_" + str(m) + "_" + str(i)]
                    totu_before += monthAnalysis["up_month_" + str(m) + "_" + str(i)] 
                totl_before /= 20
                totm_before /= 20
                totu_before /= 20
                for i in range(y+20, y+41):
                    totl_after += monthAnalysis["low_month_" + str(m) + "_" + str(i)]
                    totm_after += monthAnalysis["mid_month_" + str(m) + "_" + str(i)]
                    totu_after += monthAnalysis["up_month_" + str(m) + "_" + str(i)]                                   
                totl_after /= 20
                totm_after /= 20
                totu_after /= 20
                    
                plotTitle1 = "Average Low-Level Baroclinity ([" + mStr + "] " + str(y+40) + " - " + str(y+20) + " - " + str(y+20) + " - " + str(y) + ")"
                pathLow = p_dmov_low + monthPaths[m-1] + "dmavg_low_" + str(y+40) + "-" + str(y) + ".png"
                make_D_BCIPlot(totl_before, totl_after, plotTitle1, pathLow)
                plotTitle1 = "Average Mid-Level Baroclinity ([" + mStr + "] " + str(y+40) + " - " + str(y+20) + " - " + str(y+20) + " - " + str(y) + ")"
                pathLow = p_dmov_mid + monthPaths[m-1] + "dmavg_mid_" + str(y+40) + "-" + str(y) + ".png"
                make_D_BCIPlot(totm_before, totm_after, plotTitle1, pathLow)
                plotTitle1 = "Average Upper-Level Baroclinity ([" + mStr + "] " + str(y+40) + " - " + str(y+20) + " - " + str(y+20) + " - " + str(y) + ")"
                pathLow = p_dmov_up + monthPaths[m-1] + "dmavg_up_" + str(y+40) + "-" + str(y) + ".png"
                make_D_BCIPlot(totu_before, totu_after, plotTitle1, pathLow)   
                
                y += 20
            elif((y + 20) == 1998):
                totl_before = np.zeros((33,144))
                totm_before = np.zeros((33,144))
                totu_before = np.zeros((33,144))                
                totl_after = np.zeros((33,144))
                totm_after = np.zeros((33,144))
                totu_after = np.zeros((33,144)) 
                
                for i in range(y, y+21):
                    totl_before += monthAnalysis["low_month_" + str(m) + "_" + str(i)]
                    totm_before += monthAnalysis["mid_month_" + str(m) + "_" + str(i)]
                    totu_before += monthAnalysis["up_month_" + str(m) + "_" + str(i)] 
                totl_before /= 20
                totm_before /= 20
                totu_before /= 20
                for i in range(y+20, y+40):
                    totl_after += monthAnalysis["low_month_" + str(m) + "_" + str(i)]
                    totm_after += monthAnalysis["mid_month_" + str(m) + "_" + str(i)]
                    totu_after += monthAnalysis["up_month_" + str(m) + "_" + str(i)]                                   
                totl_after /= 19
                totm_after /= 19
                totu_after /= 19
                    
                plotTitle1 = "Average Low-Level Baroclinity ([" + mStr + "] " + str(y+39) + " - " + str(y+20) + " - " + str(y+20) + " - " + str(y) + ")"
                pathLow = p_dmov_low + monthPaths[m-1] + "dmavg_low_" + str(y+39) + "-" + str(y) + ".png"
                make_D_BCIPlot(totl_before, totl_after, plotTitle1, pathLow)
                plotTitle1 = "Average Mid-Level Baroclinity ([" + mStr + "] " + str(y+39) + " - " + str(y+20) + " - " + str(y+20) + " - " + str(y) + ")"
                pathLow = p_dmov_mid + monthPaths[m-1] + "dmavg_mid_" + str(y+39) + "-" + str(y) + ".png"
                make_D_BCIPlot(totm_before, totm_after, plotTitle1, pathLow)
                plotTitle1 = "Average Upper-Level Baroclinity ([" + mStr + "] " + str(y+39) + " - " + str(y+20) + " - " + str(y+20) + " - " + str(y) + ")"
                pathLow = p_dmov_up + monthPaths[m-1] + "dmavg_up_" + str(y+39) + "-" + str(y) + ".png"
                make_D_BCIPlot(totu_before, totu_after, plotTitle1, pathLow)   
                
                y += 20                
            else:
                break

## Misclaneous Code Blocks
Anything that I'm testing, or keeping as a reference for future additions are kept down here.

In [4]:
ftp = FTP('ftp.cdc.noaa.gov')
ftp.login()

tLink = "/Datasets/ncep.reanalysis/pressure/air.2018.nc"
hLink = "/Datasets/ncep.reanalysis/pressure/hgt.2018.nc"
uLink = "/Datasets/ncep.reanalysis/pressure/uwnd.2018.nc"
vLink = "/Datasets/ncep.reanalysis/pressure/vwnd.2018.nc" 

print("Downloading air temperature, storing in tmp.air.nc")

localfile1 = open(dataPath + "tmp.air.nc", 'wb')
ftp.retrbinary('RETR ' + tLink, localfile1.write, 1024)
localfile1.close()    

print("Downloading height, storing in tmp.hgt.nc")
localfile2 = open(dataPath + "tmp.hgt.nc", 'wb')
ftp.retrbinary('RETR ' + hLink, localfile2.write, 1024)
localfile2.close() 

print("Downloading u winds, storing in tmp.uwnd.nc")
localfile3 = open(dataPath + "tmp.uwnd.nc", 'wb')
ftp.retrbinary('RETR ' + uLink, localfile3.write, 1024)
localfile3.close() 

print("Downloading v winds, storing in tmp.vwnd.nc")
localfile4 = open(dataPath + "tmp.vwnd.nc", 'wb')
ftp.retrbinary('RETR ' + vLink, localfile4.write, 1024)
localfile4.close() 

print("Downloads complete, opening netCDF files")

aTmp = Dataset(dataPath + "tmp.air.nc")
hTmp = Dataset(dataPath + "tmp.hgt.nc")
uTmp = Dataset(dataPath + "tmp.uwnd.nc")
vTmp = Dataset(dataPath + "tmp.vwnd.nc")

outFile = dataPath + "baroclinic.test.nc"
newNCFile = Dataset(outFile, 'w', format='NETCDF4_CLASSIC')
time = newNCFile.createDimension('time', size=None) 
lat = newNCFile.createDimension('lat', 33)
lon = newNCFile.createDimension('lon', 144)  

ll_BCI = newNCFile.createVariable('BCI_925-850', np.float32, ('time','lat','lon'))
ml_BCI = newNCFile.createVariable('BCI_850-700', np.float32, ('time','lat','lon'))   
ul_BCI = newNCFile.createVariable('BCI_700-500', np.float32, ('time','lat','lon'))  
tl_BCI = newNCFile.createVariable('BCI_500-300', np.float32, ('time','lat','lon'))

T = np.squeeze(aTmp.variables['air'][:,:,2:35,:]) 
U = np.squeeze(uTmp.variables['uwnd'][:,:,2:35,:])
V = np.squeeze(vTmp.variables['vwnd'][:,:,2:35,:])
H = np.squeeze(hTmp.variables['hgt'][:,:,2:35,:])
wVel = (U**2 + V**2)**0.5

lat = np.linspace(5,85,33)
lon = np.linspace(-177.5,180,144)
p = aTmp.variables['level'][:] # Grab the isobaric levels

itime = len(aTmp.variables['time'][:]) 
ilat = len(lat)
ilon = len(lon)    

f = TwoOmega * np.sin(lat * (np.pi / 180))
# Construct a "coriolis" array of equivalent size to the other arrays
corPar = np.zeros((itime, 1, 1, ilat, ilon))
for i,l in enumerate(lat):
    corPar[:,0,0,i,:] = f[np.where(lat == l)]
#    
wVelLow = wVel[:,np.where(p==925),:,:]
wVelHigh = wVel[:,np.where(p==850),:,:]  
geoHgtLow = H[:,np.where(p==925),:,:]
geoHgtHigh = H[:,np.where(p==850),:,:]    
thetaLow = calc_theta(T[:,np.where(p==925),:,:], 925)
thetaHi = calc_theta(T[:,np.where(p==850),:,:], 850)

dLogTheta = np.log(thetaHi) - np.log(thetaLow)
dZ = geoHgtHigh - geoHgtLow
rootTerm = np.sqrt((gConst) * (dLogTheta / dZ))
outerTerm = (wVelHigh - wVelLow) / dZ  
BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000    
ll_BCI[:] = BI[:,0,0,:,:] 

#
wVelLow = wVel[:,np.where(p==850),:,:]
wVelHigh = wVel[:,np.where(p==700),:,:]  
geoHgtLow = H[:,np.where(p==850),:,:]
geoHgtHigh = H[:,np.where(p==700),:,:]    
thetaLow = calc_theta(T[:,np.where(p==850),:,:], 850)
thetaHi = calc_theta(T[:,np.where(p==700),:,:], 700)

dLogTheta = np.log(thetaHi) - np.log(thetaLow)
dZ = geoHgtHigh - geoHgtLow
rootTerm = np.sqrt((gConst) * (dLogTheta / dZ))
outerTerm = (wVelHigh - wVelLow) / dZ  
BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000    
ml_BCI[:] = BI[:,0,0,:,:]  

#
wVelLow = wVel[:,np.where(p==700),:,:]
wVelHigh = wVel[:,np.where(p==500),:,:]  
geoHgtLow = H[:,np.where(p==700),:,:]
geoHgtHigh = H[:,np.where(p==500),:,:]    
thetaLow = calc_theta(T[:,np.where(p==700),:,:], 700)
thetaHi = calc_theta(T[:,np.where(p==500),:,:], 500)

dLogTheta = np.log(thetaHi) - np.log(thetaLow)
dZ = geoHgtHigh - geoHgtLow
rootTerm = np.sqrt((gConst) * (dLogTheta / dZ))
outerTerm = (wVelHigh - wVelLow) / dZ  
BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000    
ul_BCI[:] = BI[:,0,0,:,:] 

#
wVelLow = wVel[:,np.where(p==500),:,:]
wVelHigh = wVel[:,np.where(p==300),:,:]  
geoHgtLow = H[:,np.where(p==500),:,:]
geoHgtHigh = H[:,np.where(p==300),:,:]    
thetaLow = calc_theta(T[:,np.where(p==500),:,:], 500)
thetaHi = calc_theta(T[:,np.where(p==300),:,:], 300)

dLogTheta = np.log(thetaHi) - np.log(thetaLow)
dZ = geoHgtHigh - geoHgtLow
rootTerm = np.sqrt((gConst) * (dLogTheta / dZ))
outerTerm = (wVelHigh - wVelLow) / dZ  
BI = 0.31 * (corPar / rootTerm) * (outerTerm) * 100000    
tl_BCI[:] = BI[:,0,0,:,:] 

aTmp.close()
hTmp.close()
uTmp.close()
vTmp.close()  
newNCFile.close()

print("Deleting temporary files")
os.remove(dataPath + "tmp.air.nc")
os.remove(dataPath + "tmp.hgt.nc")
os.remove(dataPath + "tmp.uwnd.nc")
os.remove(dataPath + "tmp.vwnd.nc")

ftp.quit()

Downloads complete, opening netCDF files


D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:77: RuntimeWarning: invalid value encountered in sqrt
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:79: RuntimeWarning: divide by zero encountered in divide
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:79: RuntimeWarning: invalid value encountered in multiply
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:92: RuntimeWarning: invalid value encountered in sqrt
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:94: RuntimeWarning: divide by zero encountered in divide
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:122: RuntimeWarning: invalid value encountered in sqrt
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:124: RuntimeWarning: divide by zero encountered in divide


Deleting temporary files


'221 Goodbye.'